In [1]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras import layers
from keras import optimizers
from keras.layers import Input, Lambda, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AlphaDropout
from keras.layers import Conv2D, SpatialDropout2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils import plot_model
import keras.backend as K
from keras.applications import resnet, resnet_v2, vgg16, vgg19
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
(x_tr, y_tr), (x_test_norm, y_test_norm) = mnist.load_data()
x_tr = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_tr])
x_test_norm = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_test_norm])

In [3]:
x_tr = x_tr.reshape(60000, 56, 56, 1)
x_test_norm = x_test_norm.reshape(10000, 56, 56, 1)
y_tr = to_categorical(y_tr)
y_test_norm = to_categorical(y_test_norm)

In [6]:
base_model=resnet_v2.ResNet50V2(include_top=False, input_shape = (56, 56, 1), weights=None, pooling='avg')

last = base_model.layers[-1]
second = base_model.layers[-2]

# x = SpatialDropout2D(0.5)(second.output)
# x = last(x)
x = Dense(10, activation = 'softmax')(last.output)

new_model = Model(inputs=base_model.input, outputs=x)

In [10]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum = 0.9, nesterov = True)
new_model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
new_model.fit(x_tr, y_tr, validation_data = (x_test_norm, y_test_norm), batch_size=32, epochs=2)

In [12]:
new_model.evaluate(x_test_norm, y_test_norm)

10000/10000 [==============================] - 8s 813us/step


[0.019084783305926247, 0.9930999875068665]

In [14]:
# new_model.save('models/resnetv2_99.h5')

In [4]:
new_model = load_model('models/resnetv2_99.h5')

In [6]:
import pandas as pd
attacktest = pd.read_csv('whitetoblack.csv', header=None)
attacktest = attacktest.to_numpy()
attacktest = attacktest.reshape(10000, 56, 56, 1)
new_model.evaluate(attacktest, y_test_norm)

10000/10000 [==============================] - 9s 900us/step


[0.022550889069284312, 0.9922000169754028]